In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import median_absolute_error
from sklearn.metrics import explained_variance_score

In [2]:
# Opciones de pandas
rows, cols = 50, 50
pd.set_option("display.max.columns", cols)
pd.set_option("display.max.rows", rows)
# Rutas
PATH = "../technical_daily/technical_concat/"

In [3]:
data = pd.read_csv(PATH+'testconcat.csv', header=[0,1], index_col=0)
data.set_index(pd.DatetimeIndex(list(data.index)).strftime('%Y-%m-%d'), inplace=True)
data

Attributes       Close                                                 \
Symbol             BDX         GL         IBM         NSC         PEP   
1985-03-01    1.721667   3.255194   14.186936    2.770183    1.143114   
1985-03-04    1.687380   3.033250   14.095586    2.754384    1.141212   
1985-03-05    1.655797   3.107232   14.095586    2.738583    1.164987   
1985-03-06    1.642262   3.088737   13.873695    2.759651    1.147157   
1985-03-07    1.637751   3.033250   13.586567    2.822849    1.144183   
...                ...        ...         ...         ...         ...   
2020-12-23  244.250000  92.720818  122.266174  232.182770  144.410004   
2020-12-24  247.990005  92.690872  123.045761  233.447678  145.059998   
2020-12-28  248.149994  93.110062  123.174042  235.758347  146.910004   
2020-12-29  248.350006  91.982246  122.167496  233.049286  147.419998   
2020-12-30  246.679993  93.030212  122.700371  235.031281  147.309998   

Attributes   CloseNext                                                 \
Symbol             BDX         GL         IBM         NSC         PEP   
1985-03-01    1.687380   3.033250   14.095586    2.754384    1.141212   
1985-03-04    1.655797   3.107232   14.095586    2.738583    1.164987   
1985-03-05    1.642262   3.088737   13.873695    2.759651    1.147157   
1985-03-06    1.637751   3.033250   13.586567    2.822849    1.144183   
1985-03-07    1.619704   3.005508   13.534367    2.775449    1.141212   
...                ...        ...         ...         ...         ...   
2020-12-23  247.990005  92.690872  123.045761  233.447678  145.059998   
2020-12-24  248.149994  93.110062  123.174042  235.758347  146.910004   
2020-12-28  248.350006  91.982246  122.167496  233.049286  147.419998   
2020-12-29  246.679993  93.030212  122.700371  235.031281  147.309998   
2020-12-30         NaN        NaN         NaN         NaN         NaN   

Attributes        High                                                 \
Symbol             BDX         GL         IBM         NSC         PEP   
1985-03-01    1.721667   3.292185   14.278297    2.785982    1.154899   
1985-03-04    1.718962   3.273691   14.239152    2.785983    1.153099   
1985-03-05    1.687378   3.125728   14.186946    2.754383    1.170931   
1985-03-06    1.673845   3.107232   14.147776    2.775450    1.167960   
1985-03-07    1.642263   3.088737   13.925905    2.822849    1.153099   
...                ...        ...         ...         ...         ...   
2020-12-23  247.229996  93.449405  123.558897  233.258430  145.600006   
2020-12-24  249.250000  93.379536  123.450351  233.995469  145.110001   
2020-12-28  249.869995  94.477422  124.930568  237.511265  147.600006   
2020-12-29  250.750000  93.589134  123.825342  237.411666  148.770004   
2020-12-30  249.339996  93.658994  123.203648  235.987428  147.990005   

Attributes         Low                                                 \
Symbol             BDX         GL         IBM         NSC         PEP   
1985-03-01    1.676832   3.236698   13.991164    2.743850    1.125437   
1985-03-04    1.678356   2.996260   14.030328    2.743851    1.135268   
1985-03-05    1.651285   3.051746   14.056431    2.717518    1.141212   
1985-03-06    1.642262   3.051746   13.860644    2.717519    1.144184   
1985-03-07    1.624216   3.033250   13.573515    2.759651    1.132296   
...                ...        ...         ...         ...         ...   
2020-12-23  244.179993  91.932340  122.108281  229.573308  144.070007   
2020-12-24  244.899994  91.702776  122.572087  230.698779  144.139999   
2020-12-28  247.289993  92.750755  122.818788  234.244453  145.539993   
2020-12-29  247.279999  91.902399  121.614875  231.515469  146.839996   
2020-12-30  246.589996  92.471295  121.999734  233.059245  147.009995   

Attributes        Open                                                 ...  \
Symbol             BDX         GL         IBM         NSC         PEP  ...   
1985-03-01    1.676832   0.000000   14.04337

## limpieza

En el primer box realizo una limpieza teniendo todas las acciones para que sea mas facil, ya que estas fechas en todas las acciones estan vacias por lo tanto se eliminan.

En el segundo box se borran estas acciones las cuales tienen muchos datos vacios

En el tercer box se hace uso de ffill 'forward fill' and will propagate last valid observation forward. inplace : If True, fill in place.

In [4]:
#se borran estas fechas las cuales estan vacias en todas las acciones
date_drops = ['2020-12-31','2020-12-30']
data.drop(date_drops, axis=0, errors='ignore', inplace=True)
data.set_index(pd.DatetimeIndex(list(data.index)).strftime('%Y-%m-%d'), inplace=True)

In [5]:
#borrar desde 1980
data.drop(data.index[:data.index.get_loc('1985-03-05')], axis=0, inplace=True)

In [6]:
# missing values in the data are interpolated using forward-fill (‘ffill ()’)method to propagate last valid observation forward.
data = data.fillna(method='bfill')

In [7]:
#se borran estos datos debido a la alta correlacion
attributes_to_erase = ('High','Low','Open','ma20')
for i in attributes_to_erase:
    data=data.drop(i, axis=1, level=0)

## algunos analisis individuales

In [12]:
X = data.drop(['CloseNext'], level=0, axis=1)
y = data['CloseNext']
display(X.shape, y.shape)

(9030, 55)

(9030, 5)

In [13]:
#X= data frame sin target
def dfpca(stockName, X):
    scaler = StandardScaler()
    Xsc = scaler.fit_transform(X.xs(stockName, level=1, axis=1))
    pca = PCA(n_components=6)
    principalComponents = pca.fit_transform(Xsc)
    df = pd.DataFrame(data = principalComponents, columns = ['A','B','C','D','E','F'])
    df['CloseNext'] = y[stockName].values
    print(np.sum(pca.explained_variance_ratio_))
    return df

In [14]:
stock_sel = ['NSC', 'GL', 'PEP', 'BDX', 'IBM']
dfsPCA = {}
for stock in stock_sel:
    dfsPCA[stock] = dfpca(stock, X)
dfsPCA

0.9953181739963505
0.9948416197910829
0.9946246999631714
0.9945638602167504
0.9945266362519067


{'NSC':              A         B         C         D         E         F   CloseNext
 0    -1.930782 -0.167259 -0.223482 -0.148569  0.192128  0.050573    2.759651
 1    -1.879376 -0.106457  0.464119  0.323696 -0.433290  0.019363    2.822849
 2    -1.945198 -0.294632  1.306317 -0.371097 -1.255971  0.053777    2.775449
 3    -1.798383  0.150102 -1.174912  1.183987  1.162916 -0.017514    2.764918
 4    -1.944447 -0.190950 -0.361895 -0.284985  0.331177  0.060060    2.764918
 ...        ...       ...       ...       ...       ...       ...         ...
 9025  8.654322  0.715957  0.412087 -1.718396 -0.193602 -1.282270  232.182770
 9026  8.663163  0.743997  1.858670 -1.485383  1.057050 -1.400316  233.447678
 9027  8.549992  0.701545  1.104024 -2.056345  0.219539 -1.488658  235.758347
 9028  8.564781  0.566373  1.458236 -1.767908  1.120277 -1.633668  233.049286
 9029  8.475435  0.931344 -2.335577 -1.617072 -0.999027 -1.706221  235.031281
 
 [9030 rows x 7 columns],
 'GL':              A        

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(dfsPCA['NSC'].corr(), square=True, annot=True, ax=ax)

In [17]:
sc_predict = StandardScaler()
y_scaled = sc_predict.fit_transform(dfsPCA['NSC'].values[:, dfsPCA['NSC'].columns.get_loc('CloseNext'):dfsPCA['NSC'].columns.get_loc('CloseNext')+1])
y_scaled

array([[-0.79598691],
       [-0.79469648],
       [-0.79566432],
       ...,
       [ 3.96160386],
       [ 3.90628766],
       [ 3.94675793]])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(dfsPCA['NSC'].drop(['CloseNext'], axis=1), y_scaled, test_size=0.2)
y_train = y_train.ravel()
y_test = y_test.ravel()
print(X_test)
print(y_test)
clf2 = RandomForestRegressor(bootstrap=True, n_jobs=-1,max_features=None, min_samples_leaf=1, n_estimators=80, oob_score=True)
clf2.fit(X_train,y_train)
pred = clf2.predict(X_test)
r2Score = r2_score(y_test,pred)
rmse = mean_squared_error(y_test,pred, squared=False)
mae=median_absolute_error(y_test,pred)
evs=explained_variance_score(y_test, pred)

print("r2Score:",r2Score)
print("rmse:",rmse)
print("mae:",mae)
#best possible value = 1
print("evs:",evs)



             A         B         C         D         E         F
2291 -1.601297 -0.079541 -0.704158 -0.737400  0.629737 -0.078371
1898 -1.705158 -0.050946 -0.868337 -0.777559  0.722968 -0.009572
1972 -1.603413 -0.326380 -1.226233 -0.650581  0.949356  0.024514
8374  5.368859 -2.501881 -1.402975  0.297071 -0.461339  0.581679
8585  5.827882 -1.074883  1.134796 -0.985137 -0.235814 -1.730080
...        ...       ...       ...       ...       ...       ...
5155 -0.550279 -0.860019  0.848821  0.115064 -0.665743  0.513926
2687 -1.461177 -0.378410  1.011543 -0.708210 -0.839118 -0.024356
2261 -1.534084 -0.198603 -1.190541 -0.705089  1.017795 -0.007259
3533 -1.319228  0.078251 -1.110196 -0.651784  0.871416 -0.042447
190  -1.925314 -0.352503  1.093996 -0.428253 -1.094289  0.059727

[1806 rows x 6 columns]
[-0.60824452 -0.65855759 -0.62997935 ... -0.59917061 -0.5025189
 -0.78334403]
r2Score: 0.9986141146848228
rmse: 0.03803552529731549
mae: 0.00760671516765854
evs: 0.998614132825286


In [ ]:
# Chage on attributes and no change on PricePerShare
# Reduced attributes
PRED_TRAIN = sc_predict.inverse_transform(clf.predict(X_scaled))
PRED_FUTURE = sc_predict.inverse_transform(clf.predict(X_scaled3))

# Plot parameters
START_DATE_FOR_PLOTTING = '2010-03-01'
START_INDEX = data2.index.get_loc(START_DATE_FOR_PLOTTING)
fig1,ax1 = plt.subplots(figsize=(20,10))


plt.plot(data2.index[START_INDEX:], PRED_TRAIN[START_INDEX:], color='orange', label='Training predictions')
plt.plot(data3.index[:], PRED_FUTURE, color='red', label='Predicted Stock Price')
plt.plot(data1.index[START_INDEX:], data1['CloseNext'][START_INDEX:].values, color='b', label='Actual Stock Price')

plt.axvline(x = data2.index[-1], color='green', linewidth=2, linestyle='--')

plt.grid(which='major', color='#cccccc', alpha=0.5)

plt.legend(shadow=True)
#plt.title(family='DejaVu Sans', fontsize=12)
plt.xlabel('Timeline', family='DejaVu Sans', fontsize=10)
plt.ylabel('Stock Price Value', family='DejaVu Sans', fontsize=10)
plt.xticks(rotation=45, fontsize=8)
plt.show()

print("RMSE",mean_squared_error(data2['CloseNext'], PRED_TRAIN, squared=False))